In [2]:
# Import required libraries
import pandas as pd
import numpy as np

from time import sleep
from datetime import datetime
import urllib.request
from urllib.request import urlopen
import requests

# Import selenium, website is javascript, we can't use beauitfulsoup
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [3]:
# to ignore the red warnings in the code
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Make sure to download the chromdriver with same version as your google chrome
# Find the chrome driver path in C, you can add it any place, make sure to to reachable
PATH ='C:\Program Files (x86)\chromedriver.exe'

In [5]:
# In this Script we will scrap data out of TCGplayer, it's a javascript website, so we can't use beautiful soup

In [6]:
# We set a list with stores we want to scrap, and number of pages for each one 
lists= [
'https://www.tcgplayer.com/search/yugioh/product?productLineName=yugioh&seller=a2449e6e&view=list&CardType=Main+Deck+Monster&page=',
'https://www.tcgplayer.com/search/yugioh/product?productLineName=yugioh&seller=a2449e6e&view=list&CardType=Spell&page=',
'https://www.tcgplayer.com/search/yugioh/product?productLineName=yugioh&seller=a2449e6e&view=list&CardType=Trap&page=',
'https://www.tcgplayer.com/search/yugioh/product?productLineName=yugioh&seller=a2449e6e&view=list&CardType=Extra+Deck+Monster&page=',
'https://www.tcgplayer.com/search/yugioh/product?productLineName=yugioh&seller=fe1faf49&view=list&page=',
]


page_numbers = [
117,
44,
43,
22,
154    
]

In [13]:
#  Set the wait time for the driver
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
driver = webdriver.Chrome(PATH,desired_capabilities=capa)
wait = WebDriverWait(driver, 30)

In [7]:
# Create Empty DataFrame with the needed columns
dtypes = np.dtype(
    [
        ("Card_Name", str),
        ("Card_Price", float),
        ("Market_Price", float),
        ("Shipping", float),
        ("Qty", int),
        ("Rarity", str),
        ("Pack", str),
        ("Condition", str)
    ])

df = pd.DataFrame(np.empty(0, dtype=dtypes))

In [8]:
# Check the empty data frame to make sure the columns are right
df

,Card_Name,Card_Price,Market_Price,Shipping,Qty,Rarity,Pack,Condition


In [8]:
# Def function to collect data from each page
def get_page_data():
# global df to use the same df we created before
    global df
    try:
        names = driver.find_elements_by_class_name('search-result')
        for name in names:
            cards_dict= {}
            card_name = name.find_element_by_class_name('search-result__title').text
            card_market_price = name.find_element_by_class_name('search-result__market-price--value').text[1:]
            card_rarity = name.find_element_by_class_name('search-result__rarity').text[:]
            card_subtitle = name.find_element_by_class_name('search-result__subtitle').text[:]

            listings = name.find_elements_by_class_name('listing-item')
            for listing in listings :
                card_price = listing.find_element_by_class_name('listing-item__price').text[1:]
                card_shipping_price = listing.find_element_by_class_name('shipping-messages__price').text
                card_shipping_price = card_shipping_price[3:len(card_shipping_price)-9]
                card_qty_available = listing.find_element_by_class_name('add-to-cart__available').text[3:]
                card_condition = listing.find_element_by_class_name('listing-item__condition').text

                cards_dict['Card_Name'] = card_name
                cards_dict['Card_Price'] = card_price
                cards_dict['Market_Price'] = card_market_price
                cards_dict['Shipping'] = card_shipping_price
                cards_dict['Qty'] = card_qty_available
                cards_dict['Rarity'] = card_rarity
                cards_dict['Pack'] = card_subtitle
                cards_dict['Condition'] = card_condition
    #            print (cards_dict)

                df = df.append(cards_dict, ignore_index= True)
                print(df.shape)
    
    except:
        pass

In [15]:
# Create function to loop over all pages 
def get_data_from_seller(url_page, page_nums):  
    for i in range(1, (page_nums)):
        print (i)
        driver.get(url_page + str(i))
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'shipping-messages__price')))
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'search-result__subtitle')))

        sleep(1)
        driver.execute_script("window.stop();")
        get_page_data()

In [36]:
# loop over all sellers and all pages for each store
# Save each data frame to CSV, then Clean the Dataframe
for i in range(1,len(lists)):
    print (i, 'loop, staring to gather')
    get_data_from_seller(lists[i], page_numbers[i])
    name = store[i] + '.csv'
    df.to_csv(name ,sep=',',index=False)
    df = pd.DataFrame(np.empty(0, dtype=dtypes))